Now it's been cleaned, and so we need to write the sentences to a format that can be ingested by the Dynamic Word Embeddings training script.

DWE takes as input PMI matrices for each time slice, after removing words that appear infrequently and stop words. We need to set a window size L to calculate window size, and in the Word2Vec paper, they suggest a window of 20 for small corpora.

We'll have to make a vocabulary to wordID dictionary, and do this over all words.

In [1]:
# calculate rare words

import os
from os import listdir
from os.path import isfile, join

textPath = '/Users/bkitano/Desktop/Classes/Spring_2019/thesis/corpus/cleaned_txt/'

onlyfiles = [f for f in listdir(textPath) if isfile(join(textPath, f))]

len(onlyfiles)

32830

In [2]:
# frequency distribution
import nltk
from collections import Counter
from nltk.corpus import stopwords
import time
import re

english_stopwords = stopwords.words('english')

In [8]:
import time
# do it in batches
freqDistsPath = '/Users/bkitano/Desktop/Classes/spring_2019/thesis/embeddings/freqDists/'

batchSize = 100
batches = [onlyfiles[i:i + batchSize] for i in range(0, len(onlyfiles), batchSize)]
timeElapsed = 0

lastTime = time.time()
for batchIndex in range(1, len(batches)+1):
    
    batch = batches[batchIndex-1]

    freq = Counter()
    for i in range(len(batch)):
        filename = batch[i]
        
        with open( textPath + filename, 'r+') as f:
            l = time.time()
            rawtext = f.read()
            tokens = [t for t in nltk.word_tokenize(rawtext) if t.isalpha() and t.lower() not in english_stopwords]
            print(time.time() - l)
            freq += nltk.FreqDist(tokens)
    
    with open(freqDistsPath + "batch"+ str(i) + ".csv", "a+") as f:
        for k,v in freq.most_common():
            f.write( "{}, {}\n".format(k,v) )

    batchTime = time.time() - lastTime
    timeElapsed += batchTime
    ETA = time.time() + (timeElapsed/batchIndex) * (len(batches) - batchIndex)
    ETAstring = "{}:{}:{}".format( int(ETA / 3600), int( (ETA % 3600) / 60 ), int(ETA % 60))

    print("Batch {} of {} | Batch time: {} | ETA: {}".format(batchIndex, len(batches), batchTime, ETAstring))
    lastTime = time.time()


1.8317749500274658
0.34745025634765625
0.014079093933105469
0.09024810791015625
0.00745391845703125
0.04601597785949707
0.08271002769470215
0.037316083908081055
0.07963204383850098
0.0016841888427734375
1.1622190475463867
0.02895212173461914
0.07042789459228516
0.799246072769165
0.41825199127197266
0.6186461448669434
0.01659989356994629
0.5782938003540039
0.002420186996459961
0.14334678649902344


KeyboardInterrupt: 

It may be fruitful to develop machine learning approaches to unsupervised learning of spelling variations.

Okay, now all of the freqDist have been saved as csv's, so we need to merge them into one large csv. We also need to remove all the docs that couldn't be parsed. oof i fucked that one up

In [ ]:
freqDistPath = '/Users/bkitano/Desktop/Classes/Spring_2019/thesis/embeddings/freqDists/'

freqDistFiles = [f for f in listdir(freqDistPath) if isfile(join(freqDistPath, f))]

In [ ]:
from csv import DictReader

totalFreqDist = Counter()

for freqDistFile in freqDistFiles:
    try:
        with open(freqDistPath + freqDistFile) as f:
            reader = DictReader(f, fieldnames=['word', 'count'])
            freqDist = Counter({row['word']: int(row['count']) for row in reader})
            totalFreqDist += freqDist
    except:
        print(freqDistFile)
        
print(totalFreqDist.most_common(10))

In [ ]:
with open(freqDistPath + "totalFreqDist.csv", "a+") as f:
        for k,v in totalFreqDist.most_common():
            f.write( "{}, {}\n".format(k,v) )


In [ ]:
print(totalFreqDist.most_common(10))